In [ ]:
import GRASPy as gp

Documentation can be found here - https://sebporras.github.io/GRASPy/



# Setting up a local instance of GServer

At time of writing, an instance of GServer is running on bioinfr (10.139.1.21 - port 4072). To make requests, you will need to be connected to the UQ network (on campus or *via* VPN). Note that the above IP address must be specified in the ```sendRequest``` function within ```GRASPy.client``` to run jobs using this instance.<br><br>
**However, setting up a local GServer instance is currently the preferred option.** Simply obtain a JAR file for the current version of ```bnkit```, and run the following command:<br><br>
```java -cp bnkit.jar asr.GServer```<br><br>
Note that the IP address variable in the ```sendRequest``` function within ```GRASPy.client``` should be set to "localhost"

# Performing joint reconstructions.

Step 1) Submit the job to the server. You should specify either Protein, DNA or RNA but it will try guess the sequnce type if you forget. 

In [ ]:
request = gp.JointReconstruction(aln="./example_data/joint_recon/GRASPTutorial_Final.aln",
                                nwk="./example_data/joint_recon/GRASPTutorial_Final.nwk", 
                                alphabet="Protein")

job_id = request["Job"]

Step 2) Find out where your job is in the queue or the status

In [ ]:
#queue = g_requests.PlaceInQueue(job_id)
status = gp.JobStatus(job_id)

Step 3) Retrieve your job which will have the POG graphs 

In [ ]:
graphs = gp.JobOutput(job_id)

Step 4 - Optional) Request POGs for extant sequences 

In [ ]:
extant_tree = gp.ExtantPOGTree(aln="./example_data/big_test_data/GRASPTutorial_Final.aln",
nwk="./test_data/big_test_data/GRASPTutorial_Final.nwk")


In [ ]:
extant_tree

Step 5 - Option 1) Build a POG tree from extant and ancesor POGs using both of the server outputs 

- The advantage of doing it this way is that the POGTree object will contain sequence information on the BranchPoints for ancestors AND extants 

In [ ]:
tree = gp.POGTreeFromJointReconstruction(extant_tree, graphs)

tree.branchpoints["XP_012687241.1"].seq

Step 5 - Option 2) Build a POGTree from the ancestor POG and from a nwk file string 

- ONLY ancestors will have sequence information based on the most likely symbol at each position in the sequence

In [ ]:
tree = gp.POGTreeFromJointReconstruction(nwk="./example_data/joint_recon/GRASPTutorial_Final.nwk", POG_graphs=graphs)

In [ ]:
tree.writeToNwk(file_name="test_nwk")

# Learning distributions from data 

The following instructions demonstrate how to learn a probability distribution from data. I need to add option to change some of the parameters as currently just runs on default settings. 


Step 1) Send request to the server

In [ ]:
import GRASPy as gp 

request_2 = gp.LearnLatentDistributions(nwk="./example_data/EMTrain/3_2_1_1_filt.nwk", 
                                        states=["A", "B"],                                        
                                        csv_data="./example_data/EMTrain/3_2_1_1_data.csv")

second_id = request_2["Job"]

Step 2) Check the status of your job 

In [ ]:
place = gp.PlaceInQueue(second_id)

Step 3) Retrieve your job and save the output

In [ ]:
out = gp.JobOutput(second_id)

out

Step 4) The learnt distribution can then be marginalised on an ancestor node

In [ ]:
j_distrib = out["Result"]["Distrib"]

infer = gp.MarginaliseDistOnAncestor(nwk="./example_data/EMTrain/3_2_1_1_filt.nwk", 
                        states=["A", "B"], 
                        csv_data="./example_data/EMTrain/3_2_1_1_data.csv",
                        distrib=j_distrib,
                        ancestor=0)

job_three_id = infer["Job"]

In [ ]:
infered_distribution = gp.JobOutput(job_three_id)

infered_distribution

# Inference of sequence motifs *via* evolutionary modes

This example demonstrates the learning, and subsequent utilisation of, statistical parameters for evolutionary 'modes' for motif data. Here, we focus on a single mode, with very simple example data.<br><br>
Two separate commands are communicated to the server: one for training the underlying 'phylogenetic plate' using known sequence data, and another which uses the learned distribution paraeters to infer marginal probability distributions for motif characters at an unobserved (possibly ancestral) node.

In [1]:
import GRASPy as gp

files = 'example_data/Modes/'  # example file path

### Training modes from observed motif data

***Sending the request to the server<br>***
The function ```gp.TrainModes``` takes an input phylogenetic tree, observations for some (or all) sequences (in this case, characters observed in the given motif positions of an alignment), desired number of values taken by the latent state, and other optional inputs. A JSON request is then passed to the server, and optimised mixture distribution parameters are returned.

In [2]:
# Request for training mode distribution parameters from known data (i.e. motif sequences of extants)
request = gp.TrainModes(nwk=files+'motif_eg.nwk',       # tree
                        csv_data=files+'motif_eg.csv',  # motif sequence data (from alignment)
                        n_latent=2)                     # number of values taken by the latent state

# Request will be 'queued' (but most likely run almost instantly) - server will return the job number
job_id = request["Job"]

Connecting to server...

Socket connected to localhost on IP 4072

{'Message': 'Queued', 'Job': 10}


***Obtain output for the job***<br>
The JSON output below specifies distribution parameters for amino acid characters at each motif position, conditioned on a latent state. Briefly, the number and names of latent states appear in the 'Modetypes' parameters. A set (length equal to the number of latent values) of distributions is provided for each position. These are conditioned on the latent state, and indexed corresponding to the order in which latent values appear. Residue probabilities appear in alphabetical order corresponding to their single-letter code, and concluding with the gap character **-**). We store these parameters for subsequent inference at unobserved nodes.

In [3]:
output = gp.JobOutput(job_id)
distrib = output["Result"]["Distrib"]
output

Connecting to server...

Socket connected to localhost on IP 4072



{'Job': 10,
 'Result': {'Distrib': {'Targets': [[0], [0], [0]],
   'Modetypes': [{'Size': 2, 'Values': ['A', 'B'], 'Datatype': 'String'}],
   'Nodes': [{'Condition': [['A'], ['B']],
     'Pr': [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]],
     'Variable': {'Domain': {'Predef': 'Protein with gap'},
      'Name': 'A__Feature1'},
     'Nodetype': 'CPT',
     'Index': [0, 1]},
    {'Condition': [['A'], ['B']],
     'Pr': [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]],
     'Variable': {'Domain': {'Predef': 'Protein with gap'},
      'Name': 'A__Feature2'},
     'Nodetype': 'CPT',
     'Index': [0, 1]},
    {'Condition': [['A'], ['B']],
     'Pr': [[0.5,
       0.5,
       0,
       0,
       0,
       0,
       0,
       0,
       0,
       0,
       0,
       0,
       0,
       0,
       0,
       0,
       0

### Inferring marginal probabilities of motifs for unobserved nodes

***Sending the request to the server<br>***
The function ```gp.InferModes``` takes the same input phylogenetic tree, and observations used to train modes. The distribution parameters obtained from training are also provided, as well as a list of query nodes for which sequence data should not have been observed during training. Marginal inference of latent state probabilities for each query node is performed independently.

In [4]:
request = gp.InferModesMarginal(nwk=files+'motif_eg.nwk',
                                csv_data=files+'motif_eg.csv',
                                distrib=distrib,
                                query_nodes=['N0', 'N1']  # Sequence labels if leaves, or ancestor numbers for ancestors
                               )

job_id = request["Job"]

Connecting to server...

Socket connected to localhost on IP 4072

{'Message': 'Queued', 'Job': 11}


***Obtain output for the job***<br>
The JSON output below specifies an inferred marginal probability distribution over latent states at the root node (*N0*) of our example tree. Probabilities over latent state characters are provided in the "Pr" list(s), and indexed according to the list of values ("Values" list). When multiple queries are made, distributions are listed sequentially and indexed according to the "Items" list.

In [5]:
inference = gp.JobOutput(job_id)
inference

Connecting to server...

Socket connected to localhost on IP 4072



{'Job': 11,
 'Result': {'Predict': {'Items': ['N0', 'N1'],
   'Features': ['State 0'],
   'Data': [[[{'Pr': [0.2981933243304438, 0.7018066756695562],
       'Domain': {'Size': 2, 'Values': ['A', 'B'], 'Datatype': 'String'}}],
     [{'Pr': [0.5105161855728779, 0.489483814427122],
       'Domain': {'Size': 2, 'Values': ['A', 'B'], 'Datatype': 'String'}}]]]}}}

***Calculate amino acid distributions at each position***<br>

Marginal probabilities over sequence characters at the query node can be obtained by the law of total probability.
For instance, to infer the probability of a character, $X$, at feature/position 1 ($F1$) in this example:
<br><br>
$$ P(F1=X) = P(L=A)\cdot P(F1=X|L=A) + P(L=B) \cdot P(F1=X|L=B) $$
<br>
where $L$ is the latent state value, and conditional distributions correspond to those obtained from training.<br><br>
The function ```gp.motifProbsFromInference``` takes conditional distributions (learned from training) and inferred latent state probabilities, and provides a probability distribution over the amnio acids for each motif position, for each query node.

In [6]:
probs = gp.motifProbsFromInference(distrib, inference)
probs

{'N0': {'Feature1': {'A': 0.0,
   'C': 0.0,
   'D': 0.0,
   'E': 0.0,
   'F': 0.0,
   'G': 0.2981933243304438,
   'H': 0.0,
   'I': 0.0,
   'K': 0.0,
   'L': 0.0,
   'M': 0.0,
   'N': 0.0,
   'P': 0.0,
   'Q': 0.0,
   'R': 0.0,
   'S': 0.0,
   'T': 0.0,
   'V': 0.0,
   'W': 0.0,
   'Y': 0.7018066756695562,
   '-': 0.0},
  'Feature2': {'A': 0.0,
   'C': 0.0,
   'D': 0.0,
   'E': 0.0,
   'F': 0.0,
   'G': 0.2981933243304438,
   'H': 0.0,
   'I': 0.0,
   'K': 0.0,
   'L': 0.0,
   'M': 0.0,
   'N': 0.0,
   'P': 0.7018066756695562,
   'Q': 0.0,
   'R': 0.0,
   'S': 0.0,
   'T': 0.0,
   'V': 0.0,
   'W': 0.0,
   'Y': 0.0,
   '-': 0.0},
  'Feature3': {'A': 0.8509033378347781,
   'C': 0.1490966621652219,
   'D': 0.0,
   'E': 0.0,
   'F': 0.0,
   'G': 0.0,
   'H': 0.0,
   'I': 0.0,
   'K': 0.0,
   'L': 0.0,
   'M': 0.0,
   'N': 0.0,
   'P': 0.0,
   'Q': 0.0,
   'R': 0.0,
   'S': 0.0,
   'T': 0.0,
   'V': 0.0,
   'W': 0.0,
   'Y': 0.0,
   '-': 0.0}},
 'N1': {'Feature1': {'A': 0.0,
   'C': 0.0,
 

***Charlie - below are some of the things you will probably need to implement for your analysis. These would be in addition to setting up parsing alignment data to the appropriate input file format. It would probably be helpful want to make simple examples of this here with basic data that you can extend to work with your datasets. I can help with any of this, but thought it would be better for you if you had control over how it was all done.***

## Creating holdout train/test sets from a full dataset

## Comparision between jointly inferring motif positions with *2k* latent states vs. independently inferring two positions, each with *k* latent states